# Enunciado

Gerar um texto, em português, como se fosse um atendente virtual de uma imobiliária.

Através da sintetização do texto, a AV precisa dizer ao cliente que está “ligando” o que ele pode pedir. 

São quatro áreas obrigatórias:
- Vendas
- Aluguel
- Administrativo
- Financeiro

Com o microfone, a pessoa responde uma dessas áreas e através de alguma das funções de reconhecimento de áudio, a área deverá ser identificada e, novamente por sintetização de voz, deverá gerar um áudio indicando que a ligação será transferida para a área correspondente. Por exemplo: “Ótimo, vou transferir você para um de nossos corretores de vendas.”

Deve estar em um loop. E deve haver uma palavra chave pra encerrar o loop. 

Entregar:
- Texto descrevendo o problema e a solução desenvolvida
- Código em Python
- Link para vídeo demonstrando a execução do programa

## Texto descrevendo o problema


## Código

In [ ]:
from IPython.display import HTML, Audio
import numpy as np
import speech_recognition as sr

## Vídeo